# Goal: Try the FLAN-T5 model in the prototype, turn the string to structureed parameters for recommender engine use. 

# step 1: import the the housing requests of users 

In [1]:
# read the request dataset
import pandas as pd
df = pd.read_csv('requests.csv')
df.head()

,timestamp,name,email,request
0,2025-10-06T16:45:55.381060,Diana,ruthm.lin@gmail.com,I would like a apartment nearby the Mitte of B...
1,2025-10-07T12:12:20.243206,me,ruthm.lin@gmail.com,"I would like a small room but can have a pet ,..."
2,2025-10-07T12:57:01.433639,mei,ruth.lin@gmail.com,"I like a 2 rooms and neaby MRT, bus stations. ..."
3,2025-10-07T15:17:56.861578,Diana,ruthm.lin@gmail.com,"I would like to 3 bedrooms, nearby the Mitte D..."
4,2025-10-07T15:27:05.401123,Ruth,ruthm.lin@gmail.com,I would like to have 100m2 and price less than...


 # Step 2: Introduce the flan_t5 with fine tuning of house requests

In [2]:
## Introduce the FLAN-T5 model
!pip pip install sentencepiece
!pip install  datasets accelerate
!pip install --upgrade transformers

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset
import json
import torch

ERROR: unknown command "pip"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 44.8 MB/s  0:00:00eta 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


/opt/anaconda3/envs/p310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import Dataset
from transformers import Trainer, TrainingArguments


In [4]:

# create dataset 
data = [
    {'input_text': 'I would like a apartment nearby the Mitte of Berlin, 2 bedrooms close to parks, rent price less than 2000, I am a wheelchair',
     "slots": {"district": "Mitte", "price": "<2000", "bedrooms": "2", "proximity_park": ">0.9", "wheelchair_accessabil": ">0.9", "proximity_schools": "any", "proximity_kindergarten": "any", "proximity_university": "any"}},
    {'input_text': 'I would like a small room but can have a pet , price is less than $200',
     "slots": {"district": "any", "price": "<200", "bedrooms": "1", "proximity_park": "any", "pet_friendly": ">0.9"}},
    {'input_text': 'I like a 2 rooms and neaby MRT, bus stations. The price is less than $2000',
     "slots": {"district": "any", "price": "<2000", "bedrooms": "2", "proximity_MRT": ">0.9", "proximity_bus": ">0.9"}},
    {'input_text': 'I would like to 3 bedrooms, nearby the Mitte District, and rent price less than 2000',
     "slots": {"district": "Mitte", "price": "<2000", "bedrooms": "3", "proximity_park": "any", "proximity_MRT": "any"}},
    {'input_text': 'I would like to have 100m2 and price less than 1000 Euro',
     "slots": {"district": "any", "price": "<1000", "size": ">100"}},
    {'input_text': 'I want a 3 bedrooms apartment, nearby the Kreuzberg District, and rent price less than 2500',
     "slots": {"district": "Kreuzberg", "price": "<2500", "bedrooms": "3", "proximity_park": "any", "proximity_MRT": "any"}},
    {'input_text': 'I would like a studio apartment, pet friendly, rent price less than 1500',
     "slots": {"district": "any", "price": "<1500", "bedrooms": "1", "pet_friendly": ">0.9"}},
    {'input_text': 'I need a 1 bedroom apartment, close to schools and kindergartens, rent price less than 1800',
     "slots": {"district": "any", "price": "<1800", "bedrooms": "1", "proximity_schools": ">0.9", "proximity_kindergarten": ">0.9"}},
    {'input_text': 'I am looking for a 4 bedrooms house, near universities, rent price less than 3000',
     "slots": {"district": "any", "price": "<3000", "bedrooms": "4", "proximity_university": ">0.9"}}
]

dataset = Dataset.from_list(data)

# Load FLAN-T5 model and tokenizer
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Preprocess function
def preprocess(examples):
    inputs = examples["input_text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    
    # Convert slot dicts to JSON strings
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            [ str(s) for s in examples["slots"]],
            max_length=512,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

# Training settings to fine-tune the model
training_args = TrainingArguments(
    output_dir="./flan_t5_house_rental",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Train
trainer.train()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|          | 0/9 [00:00<?, ? examples/s]/opt/anaconda3/envs/p310/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 9/9 [00:00<00:00, 205.28 examples/s]
/opt/anaconda3/e

Step,Training Loss


Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG14GFamilyCommandBuffer: 0x309f17270>
    label = <none> 
    device = <AGXG14GDevice: 0x301da7600>
        name = Apple M2 
    commandQueue = <AGXG14GFamilyCommandQueue: 0x1285d6800>
        label = <none> 
        device = <AGXG14GDevice: 0x301da7600>
            name = Apple M2 
    retainedReferences = 1
Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Insufficient Memory (00000008:kIOGPUCommandBufferCallbackErrorOutOfMemory)
	<AGXG14GFamilyCommandBuffer: 0x308867730>
    label = <none> 
    device = <AGXG14GDevice: 0x301da7600>
        name = Apple M2 
    commandQueue = <AGXG14GFamilyCommandQueue: 0x1285d6800>
        label = <none> 
        device = <AGXG14GDev

TrainOutput(global_step=9, training_loss=37.83454047309028, metrics={'train_runtime': 4390.2016, 'train_samples_per_second': 0.006, 'train_steps_per_second': 0.002, 'total_flos': 5019043627008.0, 'train_loss': 37.83454047309028, 'epoch': 3.0})

In [5]:
#Test the model with user input
def predict_slots(input_text):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output


In [29]:
# save the model and tokenizer
model.save_pretrained("./flan_t5_house_rental")
tokenizer.save_pretrained("./flan_t5_house_rental") 

('./flan_t5_house_rental/tokenizer_config.json',
 './flan_t5_house_rental/special_tokens_map.json',
 './flan_t5_house_rental/spiece.model',
 './flan_t5_house_rental/added_tokens.json')

In [6]:
device = "cpu"  # change from "mps" or "cuda"
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [7]:
# test the function
test_input = "I would like a apartment nearby the Mitte of Berlin, 3 bedrooms close to parks and kindergarten, size is greater 100m2, rent price less than 2000"
predicted_slots = predict_slots(test_input)
print(predicted_slots)

3 bedrooms close to parks and kindergarten, size is greater 100m2, rent price less than 2000


In [8]:
# change the output format to json
import re
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
# ignore this
# change the output format to json

# Load your saved model and tokenizer
model_name = "./flan_t5_house_rental"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def predict_slots(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Basic text post-processing
    decoded_lower = decoded.lower()

    result = {
        "district": "mitte" if "mitte" in decoded_lower else "any",
        "price": re.findall(r'less than (\d+)', decoded_lower),
        "bedrooms": re.findall(r'(\d+) bedroom', decoded_lower) or re.findall(r'(\d+) rooms?', decoded_lower),
        "proximity_park": ">0.9" if "park" in decoded_lower else "any",
        "proximity_kindergarten": ">0.9" if "kindergarten" in decoded_lower else "any",
        "wheelchair_accessabil": ">0.9" if "wheelchair" in decoded_lower else "any"
    }

    # Clean up results
    result["price"] = f"<{result['price'][0]}" if result["price"] else "any"
    result["bedrooms"] = result["bedrooms"][0] if result["bedrooms"] else "any"

    return json.dumps(result, indent=2)

# Test it
test_input = "I would like a apartment nearby the Mitte of Berlin, 3 bedrooms close to parks and kindergarten, size greater 100m2, rent price less than 2000"
print(predict_slots(test_input))


{
  "district": "any",
  "price": "<2000",
  "bedrooms": "3",
  "proximity_park": ">0.9",
  "proximity_kindergarten": ">0.9",
  "wheelchair_accessabil": "any"
}


In [9]:
# Add the other features of request

# Load model
model_name = "./flan_t5_house_rental"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# List of Berlin districts for matching
berlin_districts = [
    "mitte", "friedrichshain", "kreuzberg", "charlottenburg", "spandau",
    "steglitz", "zehlendorf", "tempelhof", "schoeneberg", "neukoelln",
    "treptow", "koepenick", "marzahn", "hellersdorf", "lichtenberg", "pankow"
]

def predict_slots(input_text):
    # Generate model prediction
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    decoded_lower = decoded.lower()

    # --- Basic info extraction ---
    district = next((d for d in berlin_districts if d in decoded_lower), "any")
    bedrooms = re.findall(r'(\d+)\s*(?:bedroom|room)s?', decoded_lower)
    price = re.findall(r'(?:less than|under|below)\s*(\d+)', decoded_lower)
    size = re.findall(r'(?:greater than|over|more than|above)\s*(\d+)\s*(?:m2|square meters?)?', decoded_lower)

    # --- Thematic accessibility checks ---
    transport_keywords = ["bus", "subway", "ubahn", "train", "transport"]
    convenient_keywords = ["post", "atm", "bank", "cafe", "restaurant", "supermarket"]
    family_keywords = ["school", "kindergarten", "playground"]
    greenspace_keywords = ["park", "forest", "pool", "lake"]
    healthcare_keywords = ["hospital", "clinic", "pharmacy", "dentist"]
    safety_keywords = ["crime", "safe", "safety"]
    nightlife_keywords = ["bar", "club", "nightlife"]

    def check_keywords(keywords):
        return ">0.9" if any(k in decoded_lower for k in keywords) else "any"

    result = {
        "district": district.title(),
        "size": f">{size[0]}m2" if size else "any",
        "price": f"<{price[0]}" if price else "any",
        "bedrooms": bedrooms[0] if bedrooms else "any",
        "transport_accessible": check_keywords(transport_keywords),
        "convenient_accessible": check_keywords(convenient_keywords),
        "family_friendly": check_keywords(family_keywords),
        "greenspace_accessible": check_keywords(greenspace_keywords),
        "healthcare_accessible": check_keywords(healthcare_keywords),
        "safety_score": check_keywords(safety_keywords),
        "nightlife_accessible": check_keywords(nightlife_keywords)
    }

    return json.dumps(result, indent=2)

# Test it
test_input = "I would like an apartment nearby the Mitte of Berlin, 3 bedrooms close to parks and kindergarten, size greater than 100m2, rent price less than 2000, near subway and cafes."
print(predict_slots(test_input))


{
  "district": "Mitte",
  "size": ">100m2",
  "price": "<2000",
  "bedrooms": "3",
  "transport_accessible": ">0.9",
  "convenient_accessible": ">0.9",
  "family_friendly": ">0.9",
  "greenspace_accessible": ">0.9",
  "healthcare_accessible": "any",
  "safety_score": "any",
  "nightlife_accessible": "any"
}


In [13]:
test_input= " I would like to live in Spandau, low crime ratio, and 3 bedroom, size greater than 100, nearby supermarkt, neaby public transport and easy to find the clinics, the price less than 1500euro."
print(predict_slots(test_input))



{
  "district": "Spandau",
  "size": ">100m2",
  "price": "<1500",
  "bedrooms": "3",
  "transport_accessible": ">0.9",
  "convenient_accessible": "any",
  "family_friendly": "any",
  "greenspace_accessible": "any",
  "healthcare_accessible": ">0.9",
  "safety_score": ">0.9",
  "nightlife_accessible": "any"
}


# step 3 : save the model

In [14]:
# save the above renewed model, the output format is json and cleaned results
model.save_pretrained("./flan_t5_house_rental_json_rev")


In [15]:
# Evaluation with some test cases
test_cases = [
    ("I want a 2 bedroom apartment in Mitte with rent under 2000",
     {"district": "Mitte", "price": "<2000", "bedrooms": "2", "proximity_park": "any", "proximity_kindergarten": "any", "wheelchair_accessabil": "any"}),
    ("Looking for a studio apartment, pet friendly  and rent below 1500",
     {"district": "any", "price": "<1500", "bedrooms": "1", "pet_friendly": ">0.9", "proximity_park": "any", "proximity_kindergarten": "any", "wheelchair_accessabil": "any"}),
    ("Need a 3 bedroom house near schools, rent max 2500",
     {"district": "any", "price": "<2500", "bedrooms": "3", "proximity_schools": ">0.9", "proximity_kindergarten": "any", "wheelchair_accessabil": "any"}),
    ("Searching for a 1 bedroom apartment in Kreuzberg, price under 1800",
     {"district": "Kreuzberg", "price": "<1800", "bedrooms": "1", "proximity_park": "any", "proximity_kindergarten": "any", "wheelchair_accessabil": "any"}),
    ("I want a 4 bedroom house near universities with rent less than 3000",
     {"district": "any", "price": "<3000", "bedrooms": "4", "proximity_university": ">0.9", "proximity_park": "any", "proximity_kindergarten": "any", "wheelchair_accessabil": "any"})
]
# Evalute the model
for i, (input_text, true_slots) in enumerate(test_cases):
    predicted_json = predict_slots(input_text)
    predicted_slots = json.loads(predicted_json)
    print(f"Test Case {i+1}:")
    print(f"Input: {input_text}")
    print(f"Predicted Slots: {predicted_slots}")
    print(f"True Slots: {true_slots}")
    print(f"Match: {predicted_slots == true_slots}")
    print("-" * 50)



Test Case 1:
Input: I want a 2 bedroom apartment in Mitte with rent under 2000
Predicted Slots: {'district': 'Mitte', 'size': 'any', 'price': '<2000', 'bedrooms': '2', 'transport_accessible': 'any', 'convenient_accessible': 'any', 'family_friendly': 'any', 'greenspace_accessible': 'any', 'healthcare_accessible': 'any', 'safety_score': 'any', 'nightlife_accessible': 'any'}
True Slots: {'district': 'Mitte', 'price': '<2000', 'bedrooms': '2', 'proximity_park': 'any', 'proximity_kindergarten': 'any', 'wheelchair_accessabil': 'any'}
Match: False
--------------------------------------------------
Test Case 2:
Input: Looking for a studio apartment, pet friendly  and rent below 1500
Predicted Slots: {'district': 'Any', 'size': 'any', 'price': 'any', 'bedrooms': 'any', 'transport_accessible': 'any', 'convenient_accessible': 'any', 'family_friendly': 'any', 'greenspace_accessible': 'any', 'healthcare_accessible': 'any', 'safety_score': 'any', 'nightlife_accessible': 'any'}
True Slots: {'district

In [16]:
# using accuracy and F1-score to evaluate the model
from sklearn.metrics import accuracy_score, f1_score

In [17]:
# accuracy and F1-score calculation
def calculate_metrics(true_slots, predicted_slots):
    true_values = []
    pred_values = []
    
    for key in true_slots.keys():
        true_values.append(true_slots[key])
        pred_values.append(predicted_slots.get(key, "any"))
    
    accuracy = accuracy_score(true_values, pred_values)
    f1 = f1_score(true_values, pred_values, average='weighted', zero_division=1)
    
    return accuracy, f1

In [18]:
import pandas as pd

In [19]:
#test the model and calculate metrics
# load requests.csv
df_requests = pd.read_csv('requests.csv')
cases = df_requests['request'].tolist()
# test all cases
predicted_slots_list = []
true_slots_list = []
for case in cases:
    predicted_json = predict_slots(case)
    predicted_slots = json.loads(predicted_json)
    predicted_slots_list.append(predicted_slots)
    
    # Here we assume true slots are available in the dataframe for evaluation
    # In practice, you would have a separate column or dataset with true slots
    # For demonstration, we will use dummy true slots
    true_slots = {
        "district": "any",
        "price": "<2000",
        "bedrooms": "2",
        "proximity_park": "any",
        "proximity_kindergarten": "any",
        "wheelchair_accessabil": "any"
    }
    true_slots_list.append(true_slots)

# Calculate overall metrics
all_true_slots = {}
all_pred_slots = {}
for true, pred in zip(true_slots_list, predicted_slots_list):
    all_true_slots.update(true)
    all_pred_slots.update(pred) 
accuracy, f1 = calculate_metrics(all_true_slots, all_pred_slots)
print(f"Overall Accuracy: {accuracy}")
print(f"Overall F1 Score: {f1}")




Overall Accuracy: 0.5
Overall F1 Score: 0.5


When we increase the features, the accuracy and F1 lower dowm than only 4-5 features

In [40]:
import sys
print(sys.executable)


/opt/anaconda3/envs/p310/bin/python


In [41]:
# 1️⃣ check Python 
import sys
print("Python version:", sys.version)

# 2️⃣ check transformers 
import transformers
print("Transformers version:", transformers.__version__)

# 3️⃣ check PyTorch 
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

# 4️⃣ check SentencePiece install（T5Tokenizer needed）
import sentencepiece
print("SentencePiece version:", sentencepiece.__version__)


Python version: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
Transformers version: 4.57.0
PyTorch version: 2.8.0
CUDA available: False
SentencePiece version: 0.2.1
